In [1]:
%pip install mistral_inference

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
import torch
import gc
import os

from huggingface_hub import snapshot_download, login
from pathlib import Path
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from experimentPipeline import addBaselineQuestions, basePrompt, grinchPrompt, redemptionPrompt

In [27]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [28]:
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch version: 2.7.1+cu118
CUDA available: True
CUDA version: 11.8
GPU: NVIDIA GeForce RTX 2070 SUPER
GPU Memory: 8.59 GB


In [29]:
mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 750.01it/s]


'C:\\Users\\18605\\mistral_models\\7B-Instruct-v0.3'

In [30]:
# remove model and tokenizer if they exist from memory

if 'model' in globals():
    del model
if 'tokenizer' in globals():
    del tokenizer
gc.collect()
torch.cuda.empty_cache()
print(f"GPU memory free: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

GPU memory free: 0.01 GB


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

In [31]:
model_name = "google/gemma-3-1b-it"

In [32]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    low_cpu_mem_usage=True
)

print(model.hf_device_map)

{'': 0}


In [33]:
pad_token_id = model.config.pad_token_id
if pad_token_id is None:
    pad_token_id = model.config.eos_token_id
    if isinstance(pad_token_id, list):
        pad_token_id = pad_token_id[0]

In [34]:
def runExperiment():
    baselineWithQuestions = addBaselineQuestions(basePrompt.copy())
    grinchWithQuestions = addBaselineQuestions(grinchPrompt.copy())
    redemptionWithQuestions = addBaselineQuestions(redemptionPrompt.copy())

    print("=== BASELINE PHASE ===")
    for msg in baselineWithQuestions:
        if msg["role"] == "user":
            inputs = tokenizer.apply_chat_template([baselineWithQuestions[0], msg], add_generation_prompt=True,
                                                   return_tensors="pt").to(model.device)
            attention_mask = torch.ones_like(inputs)
            outputs = model.generate(inputs, attention_mask=attention_mask, max_new_tokens=256, pad_token_id=pad_token_id)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"\nQ: {msg['content']}")
            print(f"A: {response}")
    
    print("\n\n=== GRINCH PHASE ===")
    for msg in grinchWithQuestions:
        if msg["role"] == "user":
            inputs = tokenizer.apply_chat_template([grinchWithQuestions[0], msg], add_generation_prompt=True,
                                                   return_tensors="pt").to(model.device)
            attention_mask = torch.ones_like(inputs)
            outputs = model.generate(inputs, attention_mask=attention_mask,max_new_tokens=256, pad_token_id=pad_token_id)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"\nQ: {msg['content']}")
            print(f"A: {response}")
    
    print("\n\n=== REDEMPTION PHASE ===")
    for msg in redemptionWithQuestions:
        if msg["role"] == "user":
            inputs = tokenizer.apply_chat_template([redemptionWithQuestions[0], msg], add_generation_prompt=True,
                                                   return_tensors="pt").to(model.device)
            attention_mask = torch.ones_like(inputs)
            outputs = model.generate(inputs, attention_mask=attention_mask, max_new_tokens=256, pad_token_id=pad_token_id)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"\nQ: {msg['content']}")
            print(f"A: {response}")


In [35]:
runExperiment()

=== BASELINE PHASE ===


KeyboardInterrupt: 